# Importing Libraries

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.feature_selection import SelectPercentile, chi2 
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.tree import DecisionTreeRegressor
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preparing data and Cleaning it

In [2]:
df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [3]:
df.head()

In [4]:
df.shape

In [5]:
#df.isnull().sum()
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

In [6]:
df.drop(columns=['Id', 'Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu'], inplace=True)

In [7]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

In [8]:
df.shape

In [9]:
for i in null_columns:
    print(i, df.dtypes[i])

In [10]:
mean_values = df[['LotFrontage','MasVnrArea', 'GarageYrBlt']].mean()
df[['LotFrontage','MasVnrArea', 'GarageYrBlt']] = df[['LotFrontage','MasVnrArea', 'GarageYrBlt']].fillna(value=df[['LotFrontage','MasVnrArea', 'GarageYrBlt']].mean())

In [11]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

In [12]:
df.dropna(axis=0, inplace = True)

In [13]:
#to collect the data which is object
object_data = df.select_dtypes(include=['object'])
#to convert data of object type to numbers
la = preprocessing.LabelEncoder()
for i in range(object_data.shape[1]):
    object_data.iloc[:, i] = la.fit_transform(object_data.iloc[:, i])

#concat between data none object and data object after convert it
num_data = df.select_dtypes(exclude=['object'])
data = pd.concat([object_data, num_data], axis=1)


In [14]:
data.head()

# Correlation among features

In [15]:
plt.figure(figsize=(50,50))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

# Make Polynomial Features

In [16]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2, interaction_only=True)
output_nparray = poly.fit_transform(data)
target_feature_names = ['x'.join(['{}^{}'.format(pair[0],pair[1]) for pair in tuple if pair[1]!=0]) for tuple in [zip(df.columns, p) for p in poly.powers_]]
output_df = pd.DataFrame(output_nparray, columns=target_feature_names)

# Spliting data

In [17]:
x = output_df.iloc[:, :-1]
y = output_df.iloc[:, -1]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=44, shuffle =True)

# LinearRegression Model

In [19]:
LinearRegressionModel = LinearRegression()
LinearRegressionModel.fit(X_train, y_train)

In [20]:
print('Linear Regression Train Score is : ' , LinearRegressionModel.score(X_train, y_train))
print('Linear Regression Test Score is : ' , LinearRegressionModel.score(X_test, y_test))
print('Linear Regression Coef is : ' , LinearRegressionModel.coef_)
print('Linear Regression intercept is : ' , LinearRegressionModel.intercept_)


In [21]:
y_pred_linear = LinearRegressionModel.predict(X_test)

In [22]:
print('Y predict: ',y_pred_linear[:5])
print('Y test: ', y_test[:5])

# DecisionTreeRegressor Model

In [23]:
DecisionTreeRegressorModel = DecisionTreeRegressor(max_depth = 4, random_state=33)
DecisionTreeRegressorModel.fit(X_train, y_train)

In [24]:
print('Linear Regression Train Score is : ' , DecisionTreeRegressorModel.score(X_train, y_train))
print('Linear Regression Test Score is : ' , DecisionTreeRegressorModel.score(X_test, y_test))
print('Linear Regression Coef is : ' , LinearRegressionModel.coef_)
print('Linear Regression intercept is : ' , LinearRegressionModel.intercept_)

In [25]:
y_pred_decison = DecisionTreeRegressorModel.predict(X_test)

In [26]:
print('Y predict: ',y_pred_decison[:5])
print('Y test: ', y_test[:5])